### M2.B3: Assignment: Project 2 - GEDCOM data

In [1]:
### gedcom library

from gedcom.parser import Parser

In [2]:
### List of valid tags

valid_tags = ['INDI', 'NAME', 'SEX', 'BIRT', 'DEAT', 'FAMC', 'FAMS', 'FAM', 'MARR', 'HUSB', 'WIFE', 'CHIL', 'DIV' ,'DATE',
             'HEAD', 'TRLR', 'NOTE']

### Enter your file name

file = input("Enter the file name ")


Enter the file name M1.B6_Oussama_Elhaddad.ged


In [3]:
### Extracling the file data line by line and print respectively the line, the line hierarchy 
### (<level>|<tag>|<valid|<arguments>)

def gedcom(file):
    
    parser = Parser()
    parser.parse_file(file)
    for ele in parser.get_element_list():
        
        if ( (ele.get_level() == 1 and ele.get_tag() == 'DATE') or ((ele.get_level() == 2 and ele.get_tag() == 'NAME'))):
            
            print()
        else:
            print(ele)
            if ele.get_tag() in valid_tags:
                if (ele.get_tag() == 'INDI' or ele.get_tag() == 'FAM'):  
                    print(str(ele.get_level()) +'|'+ ele.get_tag() +'|'+'Y|'+ ele.get_pointer())
                else:
                    print(str(ele.get_level()) +'|'+ ele.get_tag() +'|'+'Y|'+ ele.get_value())
            else:
                print(str(ele.get_level()) +'|'+ ele.get_tag() +'|'+'N|'+ ele.get_value())

            print('*************')
        

        
        
        
gedcom(file)

0 NOTE Oussama Elhaddad

0|NOTE|Y|Oussama Elhaddad
*************
0 HEAD

0|HEAD|Y|
*************
1 SOUR Family Echo

1|SOUR|N|Family Echo
*************
2 WWW http://www.familyecho.com/

2|WWW|N|http://www.familyecho.com/
*************
1 FILE My Family

1|FILE|N|My Family
*************

1 DEST ANSTFILE

1|DEST|N|ANSTFILE
*************
1 GEDC

1|GEDC|N|
*************
2 VERS 5.5.1

2|VERS|N|5.5.1
*************
2 FORM LINEAGE-LINKED

2|FORM|N|LINEAGE-LINKED
*************
1 SUBM @I1@

1|SUBM|N|@I1@
*************

1 SUBN

1|SUBN|N|
*************
1 CHAR UTF-8

1|CHAR|N|UTF-8
*************
0 @I1@ INDI

0|INDI|Y|@I1@
*************
1 NAME Alex /Rubin/

1|NAME|Y|Alex /Rubin/
*************
2 GIVN Alex

2|GIVN|N|Alex
*************
2 SURN Rubin

2|SURN|N|Rubin
*************
2 _MARNM Rubin

2|_MARNM|N|Rubin
*************
1 SEX M

1|SEX|Y|M
*************
1 BIRT

1|BIRT|Y|
*************
2 DATE 14 OCT 1950

2|DATE|Y|14 OCT 1950
*************
1 DEAT Y

1|DEAT|Y|Y
*************
2 DATE 11 MAY 2010

2|DATE|